In [4]:
from astropy.io import fits
from astropy.table import Table
#https://hst-docs.stsci.edu/hstdhb/4-hst-data-analysis/4-4-working-with-fits-data-in-python

import requests
import re
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import datetime

In [5]:
# prep 
date = datetime.datetime.now().strftime("%d%m%y")

parent_url = 'https://pds-geosciences.wustl.edu/m2020/urn-nasa-pds-mars2020_supercam/data_calibrated_spectra/'

folder = input('Enter root folder path for data to be saved: ')
laser_folder = f'{folder}\\LIBS RDR laser data'
spectra_folder = f'{folder}\\LIBS RDR spectra'
fits_folder = f'{folder}\\LIBS RDR fits files'

meta_path = f'{folder}\\LIBS_RDR_metadata_{date}.csv'
meta_comps_path = f'{folder}\\LIBS_RDR_metadata_w_pred_comps_{date}.csv'
spectra_path = f'{folder}\\LIBS_RDR_mean_spectra_{date}.csv'

#****spectra = pd.read_csv(spectra_path)

# first, update comps file
comps = pd.read_csv('https://pds-geosciences.wustl.edu/m2020/urn-nasa-pds-mars2020_supercam/data_derived_spectra/supercam_libs_moc.csv')
comps_path =  f'{folder}\\supercam_libs_moc_{date}.csv'
comps.to_csv(comps_path, index=False)

# drop header
comps = comps.iloc[7:].copy()
comps.columns = list(comps.iloc[0])
comps.drop(index=7, inplace=True)
comps.reset_index(inplace=True, drop=True)
comps.rename(columns={'cdr_fname':'pkey'}, inplace=True)
# add version 01 to make match meta values
comps['pkey'] = [x+'01' for x in comps.pkey]

# get page contents
page = requests.get(parent_url).text

# get sol pages
def get_sol_no(sol_page):
    return int(sol_page.split('_')[1])

def no_to_sol(sol_no):
    n_zeros = 5 - len(str(sol_no))
    sol = 'sol_'+(n_zeros*'0')+str(sol_no)
    return sol

Enter root folder path for data to be saved:  C:\Users\ytsma22c\Documents\GitHub\caltarget-accuracy\extract data from PDS\new - refresh\test


In [12]:
sol_pages = list(set(re.findall(r'sol_\d{5}', page)))
sol_pages.sort()
sol_page_nos = [get_sol_no(i) for i in sol_pages]

# find which data needs adding
def get_sols_to_add():
    global meta_path, sol_page_nos
    
    # if already exists because the run broke
    if os.path.exists(meta_path):
        meta = pd.read_csv(meta_path)
        most_recent_sol_no = max(meta['sol'])
        most_recent_sol = no_to_sol(most_recent_sol_no)
        sol_to_add = [i for i in sol_pages if i > most_recent_sol]
        sol_to_add.sort()
        
    # to initiate
    else:
        meta = pd.DataFrame(columns=['pkey',
                                     'sol',
                                     'sclock',
                                     'seq_n',
                                     'target',
                                     'location_n',
                                     'producer',
                                     'version'])
        sol_to_add = sol_pages
        
    return sol_to_add, meta

sols_to_add, meta = get_sols_to_add()

In [14]:
meta

,pkey,sol,sclock,seq_n,target,location_n,producer,version
